I worked with this notbeook inside the development data folder, so if you would like to run the code, download it and put it in the right folder (or change the paths in the notebook).

In [2]:
import pprint
import json
import codecs
from nltk.tree import Tree
from collections import defaultdict

In [3]:
# Specify path to data to be analyzed (I entered the path to the tutorial data for now, so that the code runs)

with codecs.open ('tutorial/conll16st-en-01-12-16-trial/relations.json', 'r', encoding = 'utf-8') as pdtb_file:
    

    # Assign all relations (a list) to a variable
    relations = [json.loads(x) for x in pdtb_file];

In [4]:
# Loop through relations and select only explicit relations:

# List for explicit relations:

relations_explicit = []

for relation in relations:
    if relation['Type'] == 'Explicit':
        relations_explicit.append(relation)
        
print(len(relations_explicit))
pprint.pprint(relations_explicit[0])

13
{'Arg1': {'CharacterSpanList': [[783, 877]],
          'RawText': 'Several big securities firms backed off from program '
                     'trading a few months after the 1987 crash',
          'TokenList': [[783, 790, 143, 4, 0],
                        [791, 794, 144, 4, 1],
                        [795, 805, 145, 4, 2],
                        [806, 811, 146, 4, 3],
                        [812, 818, 147, 4, 4],
                        [819, 822, 148, 4, 5],
                        [823, 827, 149, 4, 6],
                        [828, 835, 150, 4, 7],
                        [836, 843, 151, 4, 8],
                        [844, 845, 152, 4, 9],
                        [846, 849, 153, 4, 10],
                        [850, 856, 154, 4, 11],
                        [857, 862, 155, 4, 12],
                        [863, 866, 156, 4, 13],
                        [867, 871, 157, 4, 14],
                        [872, 877, 158, 4, 15]]},
 'Arg2': {'CharacterSpanList': [[883, 957]],
    

In [5]:
# Split in two lists according to arg1 in same sentence and arg1 in previous sentence:

relations_ss = []
relations_ps = []
relations_other = []

for relation in relations_explicit:
    sentence_id_arg1 = relation['Arg1']['TokenList'][0][3]
    sentence_id_connective = relation['Connective']['TokenList'][0][3]
    sentence_id_arg2 = relation['Arg2']['TokenList'][0][3]
    
    if sentence_id_arg1 == sentence_id_connective == sentence_id_arg2:
        relations_ss.append(relation)
    elif int(sentence_id_arg1) == int(sentence_id_connective) - 1 == int(sentence_id_arg2) -1:
        relations_ps.append(relation)
    else: 
        relations_other.append(relation)
print(len(relations_ps))
print(len(relations_ss))

3
9


In [6]:
# Open parse file:

with codecs.open('tutorial/conll16st-en-01-12-16-trial/parses.json', 'r', encoding = 'utf8') as parse_file:
    parses = json.load(parse_file)

In [7]:
# NOT NEEDED 

def normalize_str(text):
    """"""
    
    text = text.lstrip('(')
    
    text = text.rstrip(')')
    text_list = text.split()
    
    clean_text_list = []
    
    for word in text_list:
        word = word.strip()
        clean_text_list.append(word)
    
    clean_text = ' '.join(clean_text_list)
    
    return(clean_text)

my_rel = relations_ss[7]

arg1_str = my_rel['Arg1']['RawText']
print(arg1_str)
print(normalize_str(arg1_str))
print(normalize_str("(rtc's! Work\n didn t doesn t $50"))

We would stop index arbitrage when the market is under stress
We would stop index arbitrage when the market is under stress
rtc's! Work didn t doesn t $50


In [8]:
def get_const_type(constituent_list, constituent_name):
    """
    Input: set of constituents, name of a constituent type (str)
    Output: list of constituents of the selected type 
    """
    const_type_list = []
    
    for constituent in constituent_list:
        
        
        if constituent.startswith('('+constituent_name):
            const_type_list.append(constituent)
    
    return const_type_list
            
# E.g. get clauses:   

In [9]:
def get_subtrees(parsed_sentence, const_type = 0):
    """"""
    tree_nltk = Tree.fromstring(parsed_sentence)
    
    tree_height = tree_nltk.height()
    
        
    const_tuple_list = []

    for s in tree_nltk.subtrees():

        label = s.label()
        
        if (label != '``') and (label != ''):
            
            subtree_tokens_str = str(s.flatten())
            #print(type(subtree_tokens_str))
            clean_subtree_str = normalize_str(subtree_tokens_str).lstrip(label).lstrip(' ')



            if const_type != 0:
                if label == const_type:
                    const_tuple_list.append((label, clean_subtree_str))

            else:

                const_tuple_list.append((label, clean_subtree_str))



    return const_tuple_list

doc_id = my_rel['DocID']
sentence_id_arg1 = my_rel['Arg1']['TokenList'][0][3]
parsed_sentence = parses[doc_id]['sentences'][sentence_id_arg1]['parsetree']  

consts = get_subtrees(parsed_sentence)

#pprint.pprint(consts)

In [143]:
def get_data(relation, arg):
    
    """returns DocID, sentenceID, Argument raw text and sentence_token_id in relations.json"""
    
    relation_id = relation['ID']
    doc_id = relation['DocID']
    sentence_id_arg = relation[arg]['TokenList'][0][3]
    arg_str = relation[arg]['RawText']   
    
    sentence_token_id_relations = []
    
    relations_token_list = relation[arg]['TokenList']
    
    for line in relations_token_list:
        sentence_token_id = line[4]
        sentence_token_id_relations.append(sentence_token_id)
    
    return relation_id, doc_id, sentence_id_arg, arg_str, sentence_token_id_relations

my_rel = relations_ss[3]
my_relid, my_docid, my_sid, my_arg, my_st = get_data(my_rel, 'Arg1')
print(my_id, my_st)

for rel in relations_ss:
    print(get_data(rel, 'Arg1'))

wsj_1000 [0, 1, 2, 3, 4, 5]
(14884, 'wsj_1000', 13, "Index arbitrage doesn't work", [1, 2, 3, 4, 5])
(14885, 'wsj_1000', 14, 'he said that "on an intraday basis, it has major effects', [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34])
(14887, 'wsj_1000', 15, 'and told them to cool it', [8, 9, 10, 11, 12, 13])
(14889, 'wsj_1000', 16, 'They said, `Too bad', [0, 1, 2, 3, 4, 5])
(14895, 'wsj_1000', 22, "they'd love to do away with it {program trading}", [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18])
(14897, 'wsj_1000', 24, "the market's volatility disturbs him", [2, 3, 4, 5, 6, 7])
(14900, 'wsj_1000', 28, 'We would stop index arbitrage', [1, 2, 3, 4, 5])
(14901, 'wsj_1000', 28, 'We would stop index arbitrage when the market is under stress', [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])
(14905, 'wsj_1000', 32, 'this prompts others to consider the same thing', [2, 3, 4, 5, 6, 7, 8, 9])


In [144]:
def parse_sentence_token_id(doc_id, sentence_id_arg, sentence_token_id_relations):
    """"""
    
    argument_id_tuples_list = []
    
    parsed_words_list = parses[doc_id]['sentences'][sentence_id_arg]['words']
    
    for number, word in enumerate(parsed_words_list):
        
        if number in sentence_token_id_relations:
        
            argument_id_tuples_list.append((number, word[0]))
        
        
    return argument_id_tuples_list

my_tuple_list = parse_sentence_token_id('wsj_1000', 22, my_st)

# Looks good, if you compare these ids to the sentence_token_ids taken from relations.json. 
# Covers the span of the argument

In [145]:
def get_arg_list(argument_tuples_list):
    
    """"""


    # check if argument is interrupted by comparing to the sentence token ids in the relations.json file
    
    for index, pair in enumerate(argument_tuples_list):
        if pair == argument_tuples_list[-1]:
            break

        next_pair = argument_tuples_list[index + 1]


        if (next_pair[0] - pair[0]) > 1:
            slice_index = index
            
            break
        else:
            slice_index = None
    
    argument_tokens_as_in_parsing = []
    
    for st_id, token in argument_tuples_list:
        
        argument_tokens_as_in_parsing.append(token)  
    # Get the argument as either 1 string in a list (if it is not interrupted) or as 2 strings in a list
    # if it is interrupted

    if slice_index:
        arg1_1 = ' '.join(argument_tokens_as_in_parsing[:slice_index + 1])

        arg1_2 = ' '.join(argument_tokens_as_in_parsing[slice_index + 1:])

        arg_str_list = [arg1_1, arg1_2]
    else:

        arg_str_list = [' '.join(argument_tokens_as_in_parsing)]
    
    
    return arg_str_list

get_arg_list(my_tuple_list)

["`` The impression I 've got"]

In [147]:
# Map arguments to constituents

def constituent_structure(relation, arg):
    """"""

    # Dicts with argument as key and constituent as value (tuple: (label, constituent))
    arg_is_const = defaultdict(list)
    arg_starts_const = defaultdict(list)
    arg_in_const = defaultdict(list)
    const_part_arg = defaultdict(list)
    
    
    
    rel_id, doc_id, sentence_id_arg, arg_str, sentence_token_id_relations = get_data(relation, arg)
    
    arg_tuples_list = my_tuple_list = parse_sentence_token_id(doc_id, sentence_id_arg, sentence_token_id_relations)
    
    arg_list = get_arg_list(arg_tuples_list)


    sentence = parses[doc_id]['sentences'][sentence_id_arg]['parsetree']

    const_list = get_subtrees(sentence)


    for argument in arg_list:



        for constituent in const_list:

            label, const = constituent
    


            # Constituents the argument is part of:

            if const == argument:

                arg_is_const[argument].append((label, const))




            elif const.startswith(argument):

                arg_starts_const[argument].append((label, const))


            elif argument in const:

                arg_in_const[argument].append((label, const))



            # Constituents that are part of the argument

            if const in argument:
                const_part_arg[argument].append((label, const))
                
    return rel_id, doc_id, arg_is_const, arg_starts_const, arg_in_const, const_part_arg



In [168]:

with open('arg_constituents.tsv', 'w', encoding = 'utf-8') as outfile:   
    
    for rel in relations_ss:

        rel_id, doc_id, dict1, dict2, dict3, dict4 = constituent_structure(rel, 'Arg1')

        if dict1:
        
            for arg, const in dict1.items():
                for l, c in const:
                    print(l, c)
                    outfile.write('same sentence\t'+str(rel_id)+'\t'+str(doc_id)+'\t'+arg+'\t'+l+'\n')


S Index arbitrage does n't work
S They said , ` Too bad
SBAR they 'd love to do away with it -LCB- program trading -RCB-
S they 'd love to do away with it -LCB- program trading -RCB-
SBAR the market 's volatility disturbs him
S the market 's volatility disturbs him
S We would stop index arbitrage when the market is under stress
S this prompts others to consider the same thing


In [121]:

if dict1:
    
    print('yes')


defaultdict(<class 'list'>, {})
defaultdict(<class 'list'>, {'he said that `` on an intraday basis , it has major effects': [('NP', 'he'), ('PRP', 'he'), (',', ','), ('NP', 'he'), ('PRP', 'he'), ('VP', 'said that `` on an intraday basis , it has major effects'), ('VBD', 'said'), ('SBAR', 'that `` on an intraday basis , it has major effects'), ('IN', 'that'), ('S', 'on an intraday basis , it has major effects'), ('PP', 'on an intraday basis'), ('IN', 'on'), ('NP', 'an intraday basis'), ('DT', 'an'), ('JJ', 'intraday'), ('NN', 'basis'), (',', ','), ('NP', 'it'), ('PRP', 'it'), ('VP', 'has major effects'), ('VBZ', 'has'), ('NP', 'major effects'), ('JJ', 'major'), ('NNS', 'effects')]})


In [112]:
print(dict4)

defaultdict(<class 'list'>, {"they 'd love to do away with it -LCB- program trading -RCB-": [('SBAR', "they 'd love to do away with it -LCB- program trading -RCB-"), ('S', "they 'd love to do away with it -LCB- program trading -RCB-"), ('NP', 'they'), ('PRP', 'they'), ('VP', "'d love to do away with it -LCB- program trading -RCB-"), ('MD', "'d"), ('VP', 'love to do away with it -LCB- program trading -RCB-'), ('VB', 'love'), ('S', 'to do away with it -LCB- program trading -RCB-'), ('VP', 'to do away with it -LCB- program trading -RCB-'), ('TO', 'to'), ('VP', 'do away with it -LCB- program trading -RCB-'), ('VB', 'do'), ('ADVP', 'away with it'), ('RB', 'away'), ('PP', 'with it'), ('IN', 'with'), ('NP', 'it'), ('PRP', 'it'), ('PRN', '-LCB- program trading -RCB-'), ('-LRB-', '-LCB-'), ('NP', 'program trading'), ('NN', 'program'), ('NN', 'trading'), ('-RRB-', '-RCB-'), ('NP', 'they'), ('PRP', 'they'), ('-LRB-', '-LCB-'), ('DT', 'the'), ('-RRB-', '-RCB-'), ('VB', 'do'), ('NP', 'it'), ('PRP',

In [113]:
print(dict3)

defaultdict(<class 'list'>, {"they 'd love to do away with it -LCB- program trading -RCB-": [('S', "`` The impression I 've got is they 'd love to do away with it -LCB- program trading -RCB- , but they -LCB- the exchange -RCB- ca n't do it , '' he said ."), ('S', "The impression I 've got is they 'd love to do away with it -LCB- program trading -RCB- , but they -LCB- the exchange -RCB- ca n't do it"), ('S', "The impression I 've got is they 'd love to do away with it -LCB- program trading -RCB-"), ('VP', "is they 'd love to do away with it -LCB- program trading -RCB-")]})


In [96]:
arg = list(dict4.keys())

arg = arg[0]
arg_list = arg.split()
const_list = list(dict4.values())

const_str_list = []

for const in const_list[0]:
    label, c = const
    c = c.split()
    const_str_list.append((len(c), c, label))
    
longest_const = (max(const_str_list))  
print(arg_list)
print(longest_const)


for word_arg in arg:
    for word_const in longest_co
        if word_arg not in 
    

rest = arg.replace(longest_const[1], '')
rest = rest.strip(' ')
print(longest_const)
print(arg)
pprint.pprint(rest)

for const in const_list[0]:
    label, c = const
    if rest == c:
        print(const)

['he', 'said', 'that', '``', 'on', 'an', 'intraday', 'basis', ',', 'it', 'has', 'major', 'effects']
(12, ['said', 'that', '``', 'on', 'an', 'intraday', 'basis', ',', 'it', 'has', 'major', 'effects'], 'VP')


TypeError: Can't convert 'list' object to str implicitly